In [1]:
import numpy as np

# --- Schema de Dados para o Pipeline PEDE ---

# Lista de colunas que contêm valores numéricos formatados com vírgula (padrão BR)
COLUNAS_DECIMAIS = [
    'INDE 2024', 'INDE 23', 'INDE 22', 'INDE',
    'Cg', 'Cf', 'Ct', 
    'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 
    'Mat', 'Por', 'Ing', 
    'Rec Av1', 'Rec Av2', 'Rec Psicologia'
]

# Dicionário de tipos para carregamento otimizado (evita inferência errada)
SCHEMA_DTYPES = {
    'RA': str,                 # ID como string para evitar perda de zeros à esquerda se houver
    'Fase': str,
    'Fase Ideal': str,
    'Pedra': str,
    'Pedra 2024': str,
    'Pedra 23': str,
    'Pedra 22': str,
    'Pedra 21': str,
    'Pedra 20': str,
    'Turma': str,
    'Nome Anonimizado': str,
    'Gênero': str,
    'Instituição de ensino': str,
    'Escola': str,
    'Defasagem': float,        # Usamos float pois pode conter NaNs
    'Nº Av': float,
    'Ano ingresso': float,
    'Idade': float,
    'Ativo/ Inativo': str,
    'Avaliador1': str, 'Avaliador2': str, 'Avaliador3': str, 
    'Avaliador4': str, 'Avaliador5': str, 'Avaliador6': str,
    'Destaque IEG': str, 'Destaque IDA': str, 'Destaque IPV': str
}

# Colunas que devem ser parseadas como data
COLUNAS_DATA = ['Data de Nasc']

def converter_decimal_ptbr(valor):
    """
    Converte strings numéricas no formato PT-BR (1.000,00) para float Python.
    Trata valores nulos e erros como NaN.
    """
    if pd.isna(valor) or valor == '' or str(valor).strip() == '#N/A':
        return np.nan
        
    if isinstance(valor, (int, float)):
        return float(valor)
        
    try:
        # Remove pontos de milhar e troca vírgula decimal por ponto
        limpo = str(valor).replace('.', '').replace(',', '.')
        return float(limpo)
    except ValueError:
        return np.nan

# Criando dicionário de converters para usar no pd.read_csv
CONVERTERS_DECIMAIS = {col: converter_decimal_ptbr for col in COLUNAS_DECIMAIS}

In [2]:
# Schema de mapeamento dos nomes de colunas (Variações -> Nome Padrão do Dicionário)
# Estrutura: "Nome Padrão": ["Variação 1", "Variação 2", ...]

DICTIONARY_MAPPING = {
    "columns": {
        "Ano Ingresso": ["Ano ingresso"],
        "Data Nascimento": ["Ano nasc", "Data de Nasc"],
        "Atingiu PV": ["Atingiu PV"],
        "Avaliador 1": ["Avaliador1"],
        "Avaliador 2": ["Avaliador2"],
        "Avaliador 3": ["Avaliador3"],
        "Avaliador 4": ["Avaliador4"],
        "Avaliador 5": ["Avaliador5"],
        "Avaliador 6": ["Avaliador6"],
        "Cf": ["Cf"],
        "Cg": ["Cg"],
        "Ct": ["Ct"],
        "Defasagem": ["Defas", "Defasagem"],
        "Destaque IDA": ["Destaque IDA"],
        "Destaque IEG": ["Destaque IEG"],
        "Destaque IPV": ["Destaque IPV"],
        "Escola": ["Escola"],
        "Fase": ["Fase"],
        "Fase Ideal": ["Fase ideal", "Fase Ideal"],
        "Gênero": ["Gênero"],
        "IAA": ["IAA"],
        "IAN": ["IAN"],
        "IDA": ["IDA"],
        "Idade": ["Idade 22", "Idade"],
        "IEG": ["IEG"],
        # Mapeando qualquer INDE principal (ano corrente da planilha) para "INDE"
        "INDE": ["INDE 22", "INDE 23", "INDE 2023", "INDE 2024", "INDE"],
        "Indicado": ["Indicado"],
        "Inglês": ["Inglês", "Ing"],
        "Instituição de Ensino": ["Instituição de ensino"],
        "IPP": ["IPP"],
        "IPS": ["IPS"],
        "IPV": ["IPV"],
        "Matemática": ["Matem", "Mat"],
        "Nome": ["Nome", "Nome Anonimizado"],
        "Nº Av": ["Nº Av"],
        "Pedra 20": ["Pedra 20"],
        "Pedra 21": ["Pedra 21"],
        "Pedra 22": ["Pedra 22"],
        "Pedra 23": ["Pedra 23", "Pedra 2023"],
        "Pedra 24": ["Pedra 2024"],
        "Português": ["Portug", "Por"],
        "RA": ["RA"],
        "Rec Av1": ["Rec Av1"],
        "Rec Av2": ["Rec Av2"],
        "Rec Av3": ["Rec Av3"],
        "Rec Av4": ["Rec Av4"],
        "Rec Psicologia": ["Rec Psicologia"],
        "Turma": ["Turma"],
        "Ativo/Inativo": ["Ativo/ Inativo"]
    }
}

# Função auxiliar para renomear colunas do DF baseado no mapping
def padronizar_colunas(df, mapping):
    col_map = {}
    for padrao, variacoes in mapping['columns'].items():
        for var in variacoes:
            if var in df.columns:
                col_map[var] = padrao
    
    return df.rename(columns=col_map)

In [3]:
# Função para tratar colunas duplicadas no carregamento
def remover_colunas_duplicadas(df):
    """
    Remove colunas duplicadas de um DataFrame.
    Se houver colunas com o mesmo nome (ex: 'Ativo/ Inativo', 'Ativo/ Inativo.1'),
    mantém apenas a primeira ocorrência.
    """
    # Identifica colunas duplicadas (pelo nome original ou pandas sufixos)
    df = df.loc[:, ~df.columns.duplicated()]
    print(f"Colunas após remoção de duplicatas exatas: {df.shape[1]}")
    
    # Tratamento específico para sufixos do Pandas (.1, .2) se o usuário carregar sem mangle_dupe_cols=True
    # Mas o padrão do pandas é renomear. Vamos limpar esses nomes se forem cópias.
    cols_originais = [c.split('.')[0] for c in df.columns]
    
    # Se quiser forçar a remoção de colunas que o Pandas renomeou (ex: Coluna.1)
    # Verificamos se o "tronco" do nome já existe antes
    cols_unicas = []
    seen = set()
    cols_to_drop = []
    
    for i, col in enumerate(df.columns):
        nome_base = col.split('.')[0] # Remove sufixo .1, .2 gerado pelo pandas
        
        # Lógica: Se o nome base já foi visto E o pandas adicionou sufixo numérico
        if nome_base in seen and col != nome_base:
            # Verifica se o conteúdo é igual ao original
            if df[nome_base].equals(df[col]):
                cols_to_drop.append(col)
                # print(f"Coluna duplicada removida: {col} (cópia idêntica de {nome_base})")
            else:
                # Se conteúdo diferente, mantém mas avisa
                # print(f"Aviso: Coluna {col} tem nome similar a {nome_base} mas conteúdo diferente. Mantida.")
                pass
        else:
            seen.add(nome_base)
            
    if cols_to_drop:
        print(f"Removendo {len(cols_to_drop)} colunas duplicadas pelo Pandas: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)
        
    return df

In [4]:
# Libs
%pip install pandas
%pip install openpyxl
%pip install loguru

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from loguru import logger

 
logger.add(
    "app.log",   
    format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}",
    level="DEBUG"  # Nível de log
)
 

logger.info("This is an info message")
logger.warning("This is a warning")
logger.error("This is an error")

2026-02-16 16:39:08.508 | INFO     | __main__:<module>:11 - This is an info message
2026-02-16 16:39:08.509 | WARNING  | __main__:<module>:12 - This is a warning
2026-02-16 16:39:08.511 | ERROR    | __main__:<module>:13 - This is an error


In [6]:
import pandas as pd
import os
from loguru import logger

# Caminho do arquivo
file = "C:/Users/Angélica/Desktop/datathon/projeto_datathon/arquivos/BASE DE DADOS PEDE 2024 - DATATHON.xlsx"

# Função auxiliar de carregamento e limpeza inicial
def carregar_aba(excel_file, sheet_name):
    # Carrega dados
    df = pd.read_excel(excel_file, sheet_name=sheet_name, engine="openpyxl")
    
    # Remove espaços em branco dos nomes das colunas
    df.columns = df.columns.str.strip()
    
    # Remove colunas duplicadas imediatamente
    df = remover_colunas_duplicadas(df)
    
    # Tratamentos padrão de RA
    if 'RA' in df.columns:
        df['RA'] = df['RA'].astype(str).str.zfill(7)
        
    return df

print("--- Carregando 2022 ---")
df_2022 = carregar_aba(file, "PEDE2022")
print(f"Shape: {df_2022.shape}")

print("\n--- Carregando 2023 ---")
df_2023 = carregar_aba(file, "PEDE2023")
print(f"Shape: {df_2023.shape}")

print("\n--- Carregando 2024 ---")
df_2024 = carregar_aba(file, "PEDE2024")
print(f"Shape: {df_2024.shape}")

# Padronização de nomes (usando o mapping definido anteriormente)
# IMPORTANTE: Definir DICTIONARY_MAPPING antes de rodar isso (célula anterior)
if 'padronizar_colunas' in locals():
    print("\n--- Padronizando Colunas ---")
    df_2022 = padronizar_colunas(df_2022, DICTIONARY_MAPPING)
    df_2023 = padronizar_colunas(df_2023, DICTIONARY_MAPPING)
    df_2024 = padronizar_colunas(df_2024, DICTIONARY_MAPPING)

# Verificar intersecção após padronização
cols_comuns = set(df_2022.columns) & set(df_2023.columns) & set(df_2024.columns)
print(f"\nColunas comuns nos 3 anos: {len(cols_comuns)}")

--- Carregando 2022 ---
Colunas após remoção de duplicatas exatas: 42
Shape: (860, 42)

--- Carregando 2023 ---
Colunas após remoção de duplicatas exatas: 48
Removendo 1 colunas duplicadas pelo Pandas: ['Destaque IPV.1']
Shape: (1014, 47)

--- Carregando 2024 ---
Colunas após remoção de duplicatas exatas: 50
Removendo 1 colunas duplicadas pelo Pandas: ['Ativo/ Inativo.1']
Shape: (1156, 49)

--- Padronizando Colunas ---

Colunas comuns nos 3 anos: 40


In [9]:
# --- Padronização e Validação Comparativa entre Anos ---

def relatorio_comparativo(dfs_dict):
    """
    Compara colunas e conteúdo entre múltiplos DataFrames (anos).
    dfs_dict: dicionário {'2022': df22, '2023': df23, ...}
    """
    print("=== RELATÓRIO DE COMPARAÇÃO DE ESTRUTURA ===")
    
    anos = list(dfs_dict.keys())
    cols_sets = {ano: set(df.columns) for ano, df in dfs_dict.items()}
    
    # 1. Padronização (já aplicada antes desta chamada, mas verificando colunas resultantes)
    todas_colunas = set().union(*cols_sets.values())
    colunas_comuns = set.intersection(*cols_sets.values())
    
    print(f"Total de colunas únicas encontradas (união): {len(todas_colunas)}")
    print(f"Colunas comuns a todos os anos: {len(colunas_comuns)}")
    print("-" * 40)

    # 2. Diferenças entre anos adjacentes
    for i in range(len(anos) - 1):
        ano_a = anos[i]
        ano_b = anos[i+1]
        
        diff_a = cols_sets[ano_a] - cols_sets[ano_b]
        diff_b = cols_sets[ano_b] - cols_sets[ano_a]
        
        if not diff_a and not diff_b:
            print(f"✅ {ano_a} e {ano_b} possuem exatamente as mesmas colunas.")
        else:
            print(f"⚠️ Diferenças entre {ano_a} e {ano_b}:")
            if diff_a:
                print(f"   - Exclusivas em {ano_a}: {len(diff_a)} colunas ({', '.join(list(diff_a)[:5])}...)")
            if diff_b:
                print(f"   - Exclusivas em {ano_b}: {len(diff_b)} colunas ({', '.join(list(diff_b)[:5])}...)")
    print("-" * 40)
    
    # 3. Validação de Conteúdo (Tipos) para colunas comuns
    print("\n=== VALIDAÇÃO DE CONTEÚDO (Colunas Comuns) ===")
    divergencias_tipo = []
    
    for col in sorted(list(colunas_comuns)):
        tipos = {}
        for ano, df in dfs_dict.items():
            tipos[ano] = str(df[col].dtype)
        
        # Se houver mais de um tipo diferente para a mesma coluna
        if len(set(tipos.values())) > 1:
            divergencias_tipo.append((col, tipos))
            print(f"⚠️ Divergência de Tipo na coluna '{col}': {tipos}")
    
    if not divergencias_tipo:
        print("✅ Tipos de dados consistentes nas colunas comuns.")
        
    print("\n=== VALIDAÇÃO DE NULOS (Colunas Comuns) ===")
    # Exibir % de nulos para verificar se uma colunas "sumiu" conceitualmente (ficou vazia)
    for col in sorted(list(colunas_comuns)):
        msgs = []
        alerta = False
        for ano in anos:
            pct_null = dfs_dict[ano][col].isnull().mean() * 100
            if pct_null > 90: # Se mais de 90% for nulo, alerta
                alerta = True
            msgs.append(f"{ano}: {pct_null:.1f}%")
        
        if alerta:
            print(f"⚠️ Coluna '{col}' quase vazia em algum ano: " + " | ".join(msgs))
            
    print("\nConclusão da Validação Comparativa Finalizada.")

# Aplicando a padronização antes de comparar
if 'df_2022' in locals() and 'df_2023' in locals() and 'df_2024' in locals():
    print("Aplicando padronização de nomes...")
    df_2022_std = padronizar_colunas(df_2022, DICTIONARY_MAPPING)
    df_2023_std = padronizar_colunas(df_2023, DICTIONARY_MAPPING)
    df_2024_std = padronizar_colunas(df_2024, DICTIONARY_MAPPING)
    
    # Dicionário para a função de relatório
    dict_dfs = {
        '2022': df_2022_std,
        '2023': df_2023_std,
        '2024': df_2024_std
    }
    
    relatorio_comparativo(dict_dfs)
    
    # Atualizando as variáveis globais para as versões padronizadas (opcional)
    # df = pd.concat(dict_dfs.values(), ignore_index=True) # Se quiser concatenar já
else:
    print("DataFrames não carregados. Execute a célula de leitura primeiro.")

Aplicando padronização de nomes...
=== RELATÓRIO DE COMPARAÇÃO DE ESTRUTURA ===
Total de colunas únicas encontradas (união): 49
Colunas comuns a todos os anos: 40
----------------------------------------
⚠️ Diferenças entre 2022 e 2023:
   - Exclusivas em 2023: 2 colunas (Pedra 23, IPP...)
⚠️ Diferenças entre 2023 e 2024:
   - Exclusivas em 2023: 2 colunas (Rec Av4, Rec Av3...)
   - Exclusivas em 2024: 5 colunas (Pedra 24, Ativo/Inativo, Escola, Avaliador 6, Avaliador 5...)
----------------------------------------

=== VALIDAÇÃO DE CONTEÚDO (Colunas Comuns) ===
⚠️ Divergência de Tipo na coluna 'Atingiu PV': {'2022': 'object', '2023': 'float64', '2024': 'float64'}
⚠️ Divergência de Tipo na coluna 'Cf': {'2022': 'int64', '2023': 'float64', '2024': 'float64'}
⚠️ Divergência de Tipo na coluna 'Cg': {'2022': 'int64', '2023': 'float64', '2024': 'float64'}
⚠️ Divergência de Tipo na coluna 'Ct': {'2022': 'int64', '2023': 'float64', '2024': 'float64'}
⚠️ Divergência de Tipo na coluna 'Data Nasc

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [10]:

 
import sys
import os
import pandas as pd

# Adiciona o diretório raiz do projeto ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.utils import LoggerConfig, ApplicationLogger

class LeituraArquivos:
    def __init__(self, name: str = "LeituraArquivos", caminho: str = "data/"):
        """
        Inicializa a aplicação
        
        Args:
            name: Nome da aplicação
            caminho: Caminho padrão para os arquivos
        """
        # Configuração do logger
        self.config = LoggerConfig(
            app_name=name,
            log_dir=f"logs/{name.lower()}"
        )
        
        self.logger = ApplicationLogger(self.__class__.__name__, self.config)
        
        self.logger.logger.info(f"Aplicação {name} inicializada")
        self.caminho = caminho
        
    def ler_arquivo(self) -> pd.DataFrame:
        """
        Lê um arquivo CSV ou Excel e retorna um DataFrame
        
        Args:
            caminho: Caminho do arquivo a ser lido
            
        Returns:
            DataFrame com os dados lidos
        """
        
        self.logger.log_method_call("ler_arquivo", caminho=self.caminho)

        if not os.path.exists(self.caminho):
            self.logger.log_exception("ler_arquivo", e)
            self.logger.logger.error(f"Arquivo não encontrado: {self.caminho}") 
            raise FileNotFoundError(f"Arquivo não encontrado: {self.caminho}")
        
        self.logger.logger.info(f"Iniciando leitura de itens")

        if self.caminho.endswith('.xlsx'):
            df = pd.read_excel(self.caminho)
        else:
            self.logger.logger.error(f"Formato de arquivo não suportado: {self.caminho}")
            self.logger.log_exception("ler_arquivo", e)
            raise ValueError("Formato de arquivo não suportado. Use .xlsx ou .csv")

        self.logger.logger.info(f"Dados carregados com sucesso. Shape: {df.shape}. Colunas: {df.columns.tolist()}")
        return df

leitura = LeituraArquivos(caminho="C:\\Users\\Angélica\\Desktop\\datathon\\projeto_datathon\\arquivos\\BASE DE DADOS PEDE 2024 - DATATHON.xlsx")
df = leitura.ler_arquivo()

2026-02-16 16:41:13.025 | INFO     | __main__:__init__:26 - Aplicação LeituraArquivos inicializada
2026-02-16 16:41:13.026 | DEBUG    | src.utils:log_method_call:153 - ler_arquivo(caminho=C:\Users\Angélica\Desktop\datathon\projeto_datathon\arquivos\BASE DE DADOS PEDE 2024 - DATATHON.xlsx)
2026-02-16 16:41:13.028 | INFO     | __main__:ler_arquivo:47 - Iniciando leitura de itens


2026-02-16 16:41:13.680 | INFO     | __main__:ler_arquivo:56 - Dados carregados com sucesso. Shape: (860, 42). Colunas: ['RA', 'Fase', 'Turma', 'Nome', 'Ano nasc', 'Idade 22', 'Gênero', 'Ano ingresso', 'Instituição de ensino', 'Pedra 20', 'Pedra 21', 'Pedra 22', 'INDE 22', 'Cg', 'Cf', 'Ct', 'Nº Av', 'Avaliador1', 'Rec Av1', 'Avaliador2', 'Rec Av2', 'Avaliador3', 'Rec Av3', 'Avaliador4', 'Rec Av4', 'IAA', 'IEG', 'IPS', 'Rec Psicologia', 'IDA', 'Matem', 'Portug', 'Inglês', 'Indicado', 'Atingiu PV', 'IPV', 'IAN', 'Fase ideal', 'Defas', 'Destaque IEG', 'Destaque IDA', 'Destaque IPV']


# Validação de Dados e Modelagem Preditiva

Nesta seção, realizaremos:
1. **Validação dos Dados**: Verificação de consistência, valores nulos e distribuições.
2. **Engenharia de Features**: Preparação das variáveis para o modelo.
3. **Modelo Preditivo**: Criação de um modelo para estimar o risco de defasagem escolar.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

# Configuração de visualização
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

def validar_dados(df, ano):
    """
    Realiza validações básicas no DataFrame e retorna um relatório.
    """
    print(f"--- Relatório de Validação: {ano} ---")
    
    # 1. Verificar Nulos em Colunas Críticas
    cols_criticas = ['RA', 'Fase', 'INDE 2024', 'Pedra 2024', 'Defasagem', 'IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN']
    # Ajuste para colunas que podem variar de nome dependendo do ano (ex: INDE 2024 vs INDE 2023)
    # A padronização já deve ter cuidado disso, mas vamos garantir
    cols_existentes = [c for c in cols_criticas if c in df.columns]
    
    if not cols_existentes:
        print("⚠️ Nenhuma coluna crítica encontrada. Verifique se a padronização foi aplicada.")
        return

    nulos = df[cols_existentes].isnull().sum()
    if nulos.sum() > 0:
        print("⚠️ Atenção: Valores nulos encontrados:")
        print(nulos[nulos > 0])
    else:
        print("✅ Colunas críticas sem valores nulos.")

    # 2. Verificar Duplicidade de RA
    if 'RA' in df.columns:
        duplicados = df['RA'].duplicated().sum()
        if duplicados > 0:
            print(f"⚠️ Atenção: {duplicados} RAs duplicados encontrados.")
        else:
            print("✅ ID (RA) únicos validados.")
            
    # 3. Validar Consistência da Coluna Defasagem
    if 'Defasagem' in df.columns:
        # Converter para numérico se necessário, forçando NaN em erros
        # Usamos uma cópia para não alterar o DF original aqui se não for intenção
        defasagem_check = pd.to_numeric(df['Defasagem'], errors='coerce')
        print("Distribuição da Defasagem:")
        print(defasagem_check.value_counts().sort_index())
    
    print("-" * 30 + "\n")

# Assumindo que df_2024 já foi carregado nas células anteriores
if 'df_2024' in locals():
    validar_dados(df_2024, "2024")
else:
    print("DataFrame df_2024 não encontrado na memória.")

In [13]:
# --- Preparação para o Modelo Preditivo ---
import numpy as np

def preparar_dados_modelo(df):
    """
    Prepara o DataFrame para o treinamento do modelo.
    """
    # 1. Seleção de Features
    features = ['IAA', 'IEG', 'IPS', 'IPP', 'IDA', 'IPV', 'IAN', 'Idade']
    target = 'Defasagem'
    
    # Verificar se target existe
    if target not in df.columns:
        print(f"Erro: Coluna alvo '{target}' não encontrada no DataFrame.")
        return None, None, None

    # Validação se colunas de features existem
    cols_to_use = [col for col in features if col in df.columns]
    
    if not cols_to_use:
        print("Erro: Nenhuma feature encontrada no DataFrame.")
        return None, None, None

    data_model = df[cols_to_use + [target]].copy()
    
    # 2. Tratamento de Dados
    # Converter colunas numéricas (empties ou strings com vírgula)
    for col in cols_to_use + [target]: # Incluindo target na conversão se necessário
        if data_model[col].dtype == 'object':
            # Corrige a substituição: replace deve ser aplicado na string, e lidamos com NaN
            data_model[col] = data_model[col].astype(str).str.replace(',', '.', regex=False)
            data_model[col] = data_model[col].replace(['#N/A', 'nan', 'None', ''], np.nan)
        
        data_model[col] = pd.to_numeric(data_model[col], errors='coerce')
    
    # Tratamento do Target
    # Definindo 'Risco' como Defasagem < 0
    # Precisamos garantir que não haja NaNs no target antes de criar a classe
    data_model.dropna(subset=[target], inplace=True)
    
    data_model['Target_Risco'] = np.where(data_model['Defasagem'] < 0, 1, 0)
    
    # Remover linhas onde todas as features são nulas
    data_model.dropna(subset=cols_to_use, how='all', inplace=True)
    
    # Se sobrou dado
    if data_model.empty:
        print("Erro: DataFrame vazio após tratamento.")
        return None, None, None

    # Imputar valores faltantes nas features com a mediana
    imputer = SimpleImputer(strategy='median')
    X = imputer.fit_transform(data_model[cols_to_use])
    y = data_model['Target_Risco']
    
    return X, y, cols_to_use

if 'df_2024' in locals():
    # Modelagem
    X, y, feature_names = preparar_dados_modelo(df_2024)
    
    if X is not None and y is not None:
        # Split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Treino
        modelo = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        modelo.fit(X_train, y_train)
        
        # Predição
        y_pred = modelo.predict(X_test)
        
        # Avaliação
        print("--- Resultados do Modelo de Risco de Defasagem ---")
        print("Acurácia:", accuracy_score(y_test, y_pred))
        print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred, target_names=['Sem Risco', 'Em Risco']))
        
        # Importância das Variáveis
        importancia = pd.DataFrame({'Feature': feature_names, 'Importance': modelo.feature_importances_})
        importancia = importancia.sort_values(by='Importance', ascending=False)
        
        plt.figure(figsize=(10,6))
        sns.barplot(x='Importance', y='Feature', data=importancia, palette='viridis')
        plt.title('Importância das Variáveis na Predição do Risco')
        plt.show()
    else:
        print("Não foi possível treinar o modelo devido a erros na preparação dos dados.")
else:
    print("Carregue df_2024 para executar o modelo.")

NameError: name 'SimpleImputer' is not defined